Los datos están en un archivo .h5ad. Además tenemos dos, el de facs (citometría de flujo?) y droplet. 

In [2]:
import os
import scanpy as sc
import anndata
from urllib.request import urlretrieve
import pandas as pd

In [3]:
skin_droplet = anndata.read_h5ad("C:\matlab scripts\Tabula_Muris_Aging_Piel_Raton\Skin_droplet.h5ad")

c:\Users\andre\sc-2022\env\lib\site-packages\anndata\compat\__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
c:\Users\andre\sc-2022\env\lib\site-packages\anndata\compat\__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [4]:
skin_droplet

AnnData object with n_obs × n_vars = 4454 × 19860
    obs: 'age', 'batch', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'n_genes', 'sex', 'subtissue', 'tissue', 'tissue_free_annotation', 'n_counts', 'louvain'
    var: 'n_cells', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'louvain', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap', 'X_tsne'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

Es un anndata con los atributos que vemos arriba, donde las observaciones son las células. Sacamos la matriz de conteos. Por otro lado, la etiqueta va a ser si son células adultas (24m = 24 meses) o jóvenes (3m = 3 meses).

In [5]:
skin_droplet.obs.head()

,age,batch,cell,cell_ontology_class,cell_ontology_id,free_annotation,method,mouse.id,n_genes,sex,subtissue,tissue,tissue_free_annotation,n_counts,louvain
index,,,,,,,,,,,,,,,
AAAGATGTCGGATGTT-1-40-0-0,18m,0,MACA_18m_M_SKIN_52_AAAGATGTCGGATGTT,basal cell of epidermis,CL:0002187,NA,droplet,18-M-52,3612,male,NA,Skin,Skin,18496.0,0
AAATGCCTCAAGAAGT-1-40-0-0,18m,0,MACA_18m_M_SKIN_52_AAATGCCTCAAGAAGT,basal cell of epidermis,CL:0002187,NA,droplet,18-M-52,2867,male,NA,Skin,Skin,12716.0,0
AACTCAGAGGCAATTA-1-40-0-0,18m,0,MACA_18m_M_SKIN_52_AACTCAGAGGCAATTA,basal cell of epidermis,CL:0002187,NA,droplet,18-M-52,3808,male,NA,Skin,Skin,21047.0,0
AAGCCGCCATCGGACC-1-40-0-0,18m,0,MACA_18m_M_SKIN_52_AAGCCGCCATCGGACC,keratinocyte stem cell,CL:0002337,NA,droplet,18-M-52,533,male,NA,Skin,Skin,3699.0,4
ACGAGGAAGGTGATAT-1-40-0-0,18m,0,MACA_18m_M_SKIN_52_ACGAGGAAGGTGATAT,basal cell of epidermis,CL:0002187,NA,droplet,18-M-52,4550,male,NA,Skin,Skin,29435.0,2


In [6]:
columns_obs = skin_droplet.obs.columns
data_obs = {}
for variable in columns_obs: 
  description = skin_droplet.obs[variable].unique()
  data_obs[variable]= description
data_obs

{'age': ['18m', '21m']
 Categories (2, object): ['18m', '21m'],
 'batch': ['0']
 Categories (1, object): ['0'],
 'cell': array(['MACA_18m_M_SKIN_52_AAAGATGTCGGATGTT',
        'MACA_18m_M_SKIN_52_AAATGCCTCAAGAAGT',
        'MACA_18m_M_SKIN_52_AACTCAGAGGCAATTA', ...,
        'MACA_21m_F_SKIN_55_TTTGTCAAGACAGAGA',
        'MACA_21m_F_SKIN_55_TTTGTCAAGCTGCCCA',
        'MACA_21m_F_SKIN_55_TTTGTCATCTTGTTTG'], dtype=object),
 'cell_ontology_class': ['basal cell of epidermis', 'keratinocyte stem cell', 'epidermal cell', 'T cell', 'stem cell of epidermis']
 Categories (5, object): ['T cell', 'basal cell of epidermis', 'epidermal cell', 'keratinocyte stem cell', 'stem cell of epidermis'],
 'cell_ontology_id': ['CL:0002187', 'CL:0002337', 'CL:0000362', 'CL:0000084', 'CL:1000428']
 Categories (5, object): ['CL:0000084', 'CL:0000362', 'CL:0002187', 'CL:0002337', 'CL:1000428'],
 'free_annotation': ['NA']
 Categories (1, object): ['NA'],
 'method': ['droplet']
 Categories (1, object): ['droplet'],
 

Sacamos Y. Si son jóvenes será un 0 en la etiqueta, mientras que si son adultas se codifica con un 1. 

In [6]:
import numpy as np
Y_age = skin_facs.obs["age"]
#print(Y_age)

y = []

for i in range(len(Y_age)):

  if Y_age[i]=="24m":
    #np.append(y,[1])
    y.append(1)

  else:
    #np.append(y,[-1])
    y.append(0)

Y = np.array(y)

In [9]:
Y

array([1, 1, 1, ..., 0, 0, 0])

In [10]:
np.savetxt("C:\matlab scripts\Tabula_Muris_Aging_Piel_Raton\Y_skin.txt",Y,delimiter = ",")

Los nombres de los genes (genes_id) son:

In [13]:
skin_facs.n_vars

22899

In [17]:
names = skin_facs.var_names

In [19]:
print(names)

Index(['0610005C13Rik', '0610007C21Rik', '0610007L01Rik', '0610007N19Rik',
       '0610007P08Rik', '0610007P14Rik', '0610007P22Rik', '0610008F07Rik',
       '0610009B14Rik', '0610009B22Rik',
       ...
       'Zxdb', 'Zxdc', 'Zyg11a', 'Zyg11b', 'Zyx', 'Zzef1', 'Zzz3', 'a',
       'l7Rn6', 'zsGreen_transgene'],
      dtype='object', name='index', length=22899)


In [20]:
type(names)

pandas.core.indexes.base.Index

In [22]:
#np.savetxt("C:\matlab scripts\Tabula_Muris_Aging_Piel_Raton\genes_id.txt",names,delimiter = ",")
pd.Series(names,index=names).to_csv("C:\matlab scripts\Tabula_Muris_Aging_Piel_Raton\genes_id_skin.csv", header=False, index=False)

Ahora sacamos la matriz de conteos. 

In [ ]:
X = skin_facs.var["n_cells"].toarray().astype(np.uint8)
X.shape

In [23]:
columns_var = skin_facs.var.columns
data_var = {}
for variable in columns_var: 
  description = skin_facs.var[variable].unique()
  data_var[variable]= description
data_var

{'n_cells': array([ 2639, 43014, 25249, ..., 18827, 16584, 21272], dtype=int64),
 'means': array([6.4001128e-02, 1.0072623e+00, 3.9597169e-01, ..., 1.5586918e-03,
        3.6520475e-01, 1.6489928e-05], dtype=float32),
 'dispersions': array([2.9700527, 2.2391338, 1.8996348, ..., 2.0661294, 2.0698838,
        0.0444914], dtype=float32),
 'dispersions_norm': array([ 0.92446554, -0.4029399 , -0.5502703 , ...,  0.00385017,
         0.00767388, -2.05512   ], dtype=float32),
 'highly_variable': array([ True, False])}